In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(100))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [7]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('relu')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 45s 79ms/step - loss: 4.8244 - accuracy: 0.0181 - top3_acc: 0.0518 - top_k_categorical_accuracy: 0.0807 - val_loss: 4.2340 - val_accuracy: 0.0685 - val_top3_acc: 0.1666 - val_top_k_categorical_accuracy: 0.2346
Epoch 2/50
313/313 [==============================] - 25s 80ms/step - loss: 4.2572 - accuracy: 0.0518 - top3_acc: 0.1325 - top_k_categorical_accuracy: 0.1968 - val_loss: 3.9779 - val_accuracy: 0.1073 - val_top3_acc: 0.2380 - val_top_k_categorical_accuracy: 0.3211
Epoch 3/50
313/313 [==============================] - 24s 77ms/step - loss: 4.0490 - accuracy: 0.0752 - top3_acc: 0.1842 - top_k_categorical_accuracy: 0.2633 - val_loss: 3.8471 - val_accuracy: 0.1303 - val_top3_acc: 0.2690 - val_top_k_categorical_accuracy: 0.3574
Epoch 4/50
313/313 [==============================] - 23s 74ms/step - loss: 3.8954 - accuracy: 0.1032 - top3_acc: 0.2300 - top_k_categorical_accuracy: 0.3153 - val_loss: 3.6996 - val_accuracy: 0.1548 - val_to

In [8]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [4.637686729431152, 4.19889497756958, 4.000064849853516, 3.866473436355591, 3.764915943145752, 3.6666934490203857, 3.5829319953918457, 3.5164573192596436, 3.4361367225646973, 3.3743674755096436, 3.308612823486328, 3.2555992603302, 3.1886487007141113, 3.1250598430633545, 3.0680079460144043, 3.0102760791778564, 2.955095052719116, 2.89695143699646, 2.8450210094451904, 2.7892537117004395, 2.74092698097229, 2.690120220184326, 2.645820379257202, 2.602909803390503, 2.5529327392578125, 2.510305166244507, 2.457820415496826, 2.412214517593384, 2.3754165172576904, 2.325422525405884, 2.276616334915161, 2.2363250255584717, 2.18762469291687, 2.151061534881592, 2.1057353019714355, 2.061483144760132, 2.0241153240203857, 1.9866318702697754, 1.9312798976898193, 1.8967260122299194, 1.8507988452911377, 1.805557370185852, 1.7684029340744019, 1.7239269018173218, 1.6843966245651245, 1.6447759866714478, 1.595732569694519, 1.5516139268875122, 1.5186333656311035, 1.4669721126556396], 'accuracy': [0.027